## Configuration and Exploratory Analysis of the SPLINK Model

This notebook contains the configuration and exploratory analysis for using the [SPLINK](https://moj-analytical-services.github.io/splink/index.html) model to run a record linkage algorithm.
The goal is to execute this model after receiving results from [OpenSearch](https://opensearch.org/).

In general, the idea is to perform the following steps when someone queries the `GET /v1/production-locations/` endpoint with the name and address of the production location:

1. First, query the OpenSearch cluster's `production-locations` index using [hybrid search](https://opensearch.org/docs/latest/vector-search/ai-search/hybrid-search/index/).

2. After obtaining the search results, use the [real-time record linkage](https://moj-analytical-services.github.io/splink/demos/examples/duckdb/real_time_record_linkage.html) capability of the SPLINK library to generate confidence scores for potential matches.
